In [1]:
!pip install PyPDF2

In [2]:
import PyPDF2

In [3]:
from PyPDF2 import PdfReader

In [4]:
import re

In [5]:
!pip install transformers

In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [7]:
!pip install pyttsx3

In [8]:
import pyttsx3
from IPython.display import Audio

In [9]:
!pip install --upgrade gradio

In [10]:
import gradio as gr

In [11]:
!pip install pydub

In [12]:
from pydub import AudioSegment
import io

In [13]:
def read_pdf_summarize_and_speak(pdf_path):
    pdf_reader=PdfReader(pdf_path)
    content=""
    for page in pdf_reader.pages:
        text=page.extract_text()
        content += text + "\n"
    
    # Pattern to match in-text citations (e.g., [1], [2, 3], or (Author, Year))
    in_text_citations_pattern = r'\[\d+(,\s*\d+)*\]|\([A-Za-z ]+,\s*\d{4}\)'

    # Pattern to match the References section at the end of the document
    references_section_pattern = r'\nReferences[\s\S]*'

    # Remove in-text citations
    text_without_citations = re.sub(in_text_citations_pattern, '', content)

    # Remove the References section
    text_without_references = re.sub(references_section_pattern, '', text_without_citations)

    # The new variable without the references
    cleaned_text = text_without_references

    model_name = 't5-large'
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    to_tokenize = "summarize: " + content
    inputs = tokenizer.encode(to_tokenize, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=48, length_penalty=2, num_beams=5, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summary_text=summary.capitalize()
    engine = pyttsx3.init()
    engine.save_to_file(summary_text, 'summary.mp3')
    engine.runAndWait()  # Blocks while processing all currently queued commands
    Audio('summary.mp3',autoplay=True)
    


In [14]:
pdf_file_path = "C:/Users/styli/OneDrive/Desktop/OPIT/AAI/Assessment 3/Article 11 Hidden Technical Debt in Machine Learning Systems.pdf"

In [15]:
read_pdf_summarize_and_speak(pdf_file_path)

C:\Users\styli\miniconda3\envs\pydata-book\lib\site-packages\transformers\models\t5\tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. 

In [21]:
iface = gr.Interface(
    fn=read_pdf_summarize_and_speak,
    inputs=gr.File(label="Upload PDF"),
    outputs=gr.Audio(label="Speech Summary"),
    title="PDF to Speech Summarizer",
    description="Upload a PDF document to receive a one-sentence summarized version in audio format."
)


In [22]:
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7864

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
